In [1]:
import torch
from torchvision.models import resnet18
import pickle
import numpy as np
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names

In [4]:
np.sqrt(0.02*32*32*3)

7.83836717690617

In [40]:
x = torch.ones(10)
y = x + 1

In [36]:
e = torch.ones_like(x)

In [38]:
e

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [37]:
torch.linalg.vector_norm(e, ord=2)

tensor(3.1623)

In [31]:
image = x + e


In [49]:
MSE = torch.nn.MSELoss(reduction='none')

In [50]:
torch.sqrt(MSE(x,y))

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [20]:
def tanh_space(w):
    return (1/2) * (torch.tanh(w)+1)
def inverse_tanh_space(x):
    return atanh(torch.clamp(x * 2 - 1, min=-1, max=1))
def atanh(x):
    return 0.5 * torch.log((1 + x) / (1 - x))


In [23]:
images = torch.rand(size=(10, 3,32,32))
images = torch.clamp(images, min=0.2, max=0.8)

S_perturb = torch.rand_like(images)
S_perturb = torch.clamp(S_perturb, min=0.05, max=0.1)

In [24]:
images_perturbed = images + S_perturb
w = inverse_tanh_space(images_perturbed)
#delta_perturb = inverse_tanh_space(S_perturb)
#w_prime = w + delta_perturb

In [25]:
t_adv_images = tanh_space(w)

In [26]:
t_adv_images - S_perturb

tensor([[[[0.2000, 0.3501, 0.2000,  ..., 0.3975, 0.2547, 0.2000],
          [0.3698, 0.5202, 0.5185,  ..., 0.5770, 0.4540, 0.3850],
          [0.8000, 0.8000, 0.7926,  ..., 0.4291, 0.6379, 0.2087],
          ...,
          [0.8000, 0.8000, 0.2000,  ..., 0.8000, 0.7705, 0.4287],
          [0.2000, 0.2494, 0.2000,  ..., 0.2000, 0.8000, 0.2000],
          [0.6053, 0.4488, 0.5180,  ..., 0.7494, 0.2000, 0.8000]],

         [[0.6246, 0.5530, 0.8000,  ..., 0.8000, 0.6083, 0.3835],
          [0.2607, 0.3115, 0.8000,  ..., 0.7520, 0.7660, 0.5680],
          [0.4913, 0.8000, 0.4516,  ..., 0.2766, 0.5614, 0.7902],
          ...,
          [0.4151, 0.2000, 0.2000,  ..., 0.8000, 0.7982, 0.4071],
          [0.2000, 0.6126, 0.2000,  ..., 0.4151, 0.8000, 0.2000],
          [0.7271, 0.8000, 0.2659,  ..., 0.2629, 0.7909, 0.8000]],

         [[0.4150, 0.2000, 0.2302,  ..., 0.5994, 0.2445, 0.5177],
          [0.5618, 0.8000, 0.2000,  ..., 0.7635, 0.2000, 0.4453],
          [0.4752, 0.2000, 0.2000,  ..., 0

In [27]:
images

tensor([[[[0.2000, 0.3501, 0.2000,  ..., 0.3975, 0.2547, 0.2000],
          [0.3698, 0.5202, 0.5185,  ..., 0.5770, 0.4540, 0.3850],
          [0.8000, 0.8000, 0.7926,  ..., 0.4291, 0.6379, 0.2087],
          ...,
          [0.8000, 0.8000, 0.2000,  ..., 0.8000, 0.7705, 0.4287],
          [0.2000, 0.2494, 0.2000,  ..., 0.2000, 0.8000, 0.2000],
          [0.6053, 0.4488, 0.5180,  ..., 0.7494, 0.2000, 0.8000]],

         [[0.6246, 0.5530, 0.8000,  ..., 0.8000, 0.6083, 0.3835],
          [0.2607, 0.3115, 0.8000,  ..., 0.7520, 0.7660, 0.5680],
          [0.4913, 0.8000, 0.4516,  ..., 0.2766, 0.5614, 0.7902],
          ...,
          [0.4151, 0.2000, 0.2000,  ..., 0.8000, 0.7982, 0.4071],
          [0.2000, 0.6126, 0.2000,  ..., 0.4151, 0.8000, 0.2000],
          [0.7271, 0.8000, 0.2659,  ..., 0.2629, 0.7909, 0.8000]],

         [[0.4150, 0.2000, 0.2302,  ..., 0.5994, 0.2445, 0.5177],
          [0.5618, 0.8000, 0.2000,  ..., 0.7635, 0.2000, 0.4453],
          [0.4752, 0.2000, 0.2000,  ..., 0

In [ ]:
xlayer = model.layer1[1].conv1
scale_factor = 2
kernel_vectors = xlayer.weight.flatten(1)
l2_distances = torch.cdist(kernel_vectors, kernel_vectors).triu().unsqueeze(0)
l2_blocks = torch.nn.functional.unfold(l2_distances, kernel_size=scale_factor, stride=scale_factor, padding=0).T
step_size = (l2_distances.shape[1] // (scale_factor )) + 1
extract_indices = torch.arange(0, l2_blocks.shape[0], step=step_size)
repulsion_loss = -l2_blocks[extract_indices].sum()

In [ ]:
l2_blocks[extract_indices].sum(dim=1)

In [ ]:
repulsion_loss

In [ ]:
scale_factor=2

In [ ]:
tot_repulsion_loss = 0
for name, param in model.named_modules():
    if "conv" in name and "layer" in name:
        kernel_vectors = param.weight.flatten(1)
        l2_distances = torch.cdist(kernel_vectors, kernel_vectors).triu().unsqueeze(0)
        l2_blocks = torch.nn.functional.unfold(l2_distances, kernel_size=scale_factor, stride=scale_factor, padding=0).T
        step_size = (l2_distances.shape[1] // (scale_factor )) + 1
        extract_indices = torch.arange(0, l2_blocks.shape[0], step=step_size)
        repulsion_loss = -l2_blocks[extract_indices].sum()
        print(repulsion_loss)
        tot_repulsion_loss+=repulsion_loss    

In [ ]:
tot_repulsion_loss*0.0005

In [ ]:

step = 2
images = torch.randn((20,10,10))
batch_size = images.shape[0]
condition = torch.randn((batch_size,)) > 0
mask = condition
binary_success = condition.to(dtype=int)
cache_size =  10
success_thres = 0.5


In [ ]:
images = torch.randn((20,10,10))
batch_size = images.shape[0]
condition = torch.randn((batch_size,)) > 0
mask = condition
binary_success = condition.to(dtype=int)
cache_size =  10
success_thres = 0.5

moving_results = torch.zeros(size=(batch_size, cache_size))
moving_results[:] = torch.nan
moving_avg = torch.zeros(size=(batch_size,))
best_moving_avg = torch.negative(torch.ones(size=(batch_size,)))

In [ ]:
cache_idx = step % cache_size
moving_results[:, cache_idx] = binary_success
moving_avg = torch.nanmean(moving_results, dim=1)

# Update images with moving_avg


In [ ]:
mask_mov_avg = moving_avg > best_moving_avg
mask = torch.logical_and(mask_mov_avg, mask)

In [ ]:
moving_results

In [ ]:
moving_avg

In [23]:
x = ["┌─","─┐", "─┬─", "└─", "─┘", "─┴─", "├─", "─┤", "─┼─" ]


In [15]:
for el in x:
    print(el)
    for ch in el:
        print(ord(ch)) 
    print("\n\n")

NameError: name 'x' is not defined

In [56]:
loss = 2.65
all_repulsion_losses = torch.tensor([-100, -90, -80, -70, -60, -50, -40, -30]).to(dtype=torch.float32)
lam = torch.tensor([i*4 for i in range(1,len(all_repulsion_losses)+1)]).to(dtype=torch.float32)
total_repulsion_loss = torch.dot(all_repulsion_losses, lam)

In [57]:
tot_repulsion_loss_c = torch.negative(total_repulsion_loss.clone())
lam_i = 1
while tot_repulsion_loss_c > loss:
    tot_repulsion_loss_c = tot_repulsion_loss_c / 10
    lam_i*=1/10


In [58]:
lam

tensor([ 4.,  8., 12., 16., 20., 24., 28., 32.])

In [59]:
lam = torch.tensor([lam[i]*lam_i for i in range(len(lam))])

In [54]:
total_repulsion_loss = torch.dot(all_repulsion_losses, lam)

In [55]:

total_repulsion_loss

tensor(-0.7680)

In [49]:
loss

2.65

In [2]:
lam = 1
rep_loss_c = rep_loss
while rep_loss_c > loss:
    lam*=1/10
    rep_loss_c= rep_loss_c / 10